# Make injections
...according to a config file that the user can customize.

Before running this notebook, make a ``config.py`` file like the one in ``cogwheel/validation/example/config.py`` and put it in some directory of your choice. Note that eventually a lot of data will be saved to that directory (a parameter estimation run for each injection).

In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["VECLIB_MAXIMUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"

import sys
path_to_cogwheel = '../..'
sys.path.append(path_to_cogwheel)

import lal
lal.swig_redirect_standard_output_error(False)

import pandas as pd

from cogwheel.validation import load_config
from cogwheel.validation import generate_injections

from cogwheel import gw_plotting

In [ ]:
%matplotlib widget

In [ ]:
config_filename = 'example/config.py'  # Edit as appropriate
config = load_config(config_filename)

In [ ]:
generate_injections.generate_injections_from_config(config, n_cores=8)

Check the generated injections:

In [ ]:
generate_injections.test_h_h_distribution(config)

In [ ]:
injections = pd.read_feather(config.INJECTION_DIR/config.INJECTIONS_FILENAME)

# Corner plot:
params = [
    'm1',
    'm2',
    's1z',
    's2z',
    's1x_n',
    's1y_n',
    's2x_n',
    's2y_n',
    'iota',
    'ra',
    'dec',
    'psi',
    'phi_ref',
    'd_luminosity']

gw_plotting.CornerPlot(injections, params=params
                      ).plot(title=f'{config.INJECTION_DIR.as_posix()}')